In [4]:
import pandas as pd

# 读取文件
def read_excel_file(file_path):
    """
    读取 Excel 文件中的 Sheet1，返回 DataFrame
    """
    excel_file = pd.ExcelFile(file_path)
    df = excel_file.parse('Sheet1')
    return df

# 处理缺失值
def handle_missing_values(df):
    """
    使用均值填充瞬时机械钻速的缺失值
    """
    df['瞬时机械钻速(m/h)'].fillna(df['瞬时机械钻速(m/h)'].mean(), inplace=True)
    return df

# 判断并校正裂缝标签
def check_and_correct(row_index, df, window_size=6):
    """
    根据钻压、钻速和立管压力的突变判断标签是否合理，若不合理则与更符合条件的附近行互换，并标记是否被校正
    """
    start_index = max(0, row_index - window_size)
    end_index = min(len(df), row_index + window_size + 1)

    for i in range(start_index, end_index):
        if i != row_index:
            current_row = df.iloc[i]
            target_row = df.iloc[row_index]

            pressure_change = abs(current_row['钻压(kN)'] - target_row['钻压(kN)'])
            speed_change = current_row['瞬时机械钻速(m/h)'] - target_row['瞬时机械钻速(m/h)']
            pressure_drop = target_row['立管压力(MPa)'] - current_row['立管压力(MPa)']

            if (pressure_change > 1 and speed_change > 0.5 and pressure_drop > 0.3):
                # 互换标签
                df.at[i, '是否存在裂缝'], df.at[row_index, '是否存在裂缝'] = \
                    df.at[row_index, '是否存在裂缝'], df.at[i, '是否存在裂缝']
                # 添加校正标记
                df.at[i, '是否被校正'] = True
                df.at[row_index, '是否被校正'] = True
                return True
    return False

# 执行校正
def calibrate_data(df):
    """
    遍历所有裂缝标记为“有”的样本行，尝试纠正错误标签，并记录校正标记
    """
    df['是否被校正'] = False  # 初始化标记列
    for index, row in df[df['是否存在裂缝'] == '有'].iterrows():
        check_and_correct(index, df)
    return df

# 保存结果
def save_calibrated_data(df, output_file_path):
    """
    保存为 Excel 文件
    """
    df.to_excel(output_file_path, index=False)

# 主流程
def main():
    input_file_path = '训练数据_未校准.xlsx'
    output_file_path = '训练数据_校准后_含校正标记.xlsx'

    df = read_excel_file(input_file_path)
    df = handle_missing_values(df)
    df = calibrate_data(df)
    save_calibrated_data(df, output_file_path)

if __name__ == "__main__":
    main()


C:\Users\董洺瑀\AppData\Local\Temp\ipykernel_14984\430495315.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['瞬时机械钻速(m/h)'].fillna(df['瞬时机械钻速(m/h)'].mean(), inplace=True)
